In [1]:
import pandas as pd
import json
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, Cupid, DistributionBased
import numpy as np
import os

### TPC-DI

представим в виде (поскольку нет описания датасетов и атрибутов): dataset name 1, dataset name 2, attribute 1, attribute 2, attribute_match (0 or 1), constraints and meta-information

In [11]:
all_data_name1 = 'prospect_horizontal_50_ec_av'
data_name1 = all_data_name1+'_source'
data_name2 = all_data_name1+'_target'
df1 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}'.format(all_data_name1,data_name1+'.csv'))
df2 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}'.format(all_data_name1,data_name2+'.csv'))

with open(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}_mapping.json'.format(all_data_name1,all_data_name1), 'r') as f:
    mapping = json.load(f)
ground_t1 = []
for i in mapping['matches']:
    ground_t1.append((i['source_column'],i['target_column']))

In [3]:
df1

,AgencyID,LastName,FirstName,MiddleInitial,Gender,AddressLine1,AddressLine2,PostalCode,City,State,...,Income,NumberCars,NumberChildren,MaritalStatus,Age,CreditRating,OwnOrRentFlag,Employer,NumberCreditCards,NetWorth
0,PEL0,PELLAND,Netti,NaN,f,21847 olympia street,NaN,T6b 1i1,Fairbanks,MA,...,368776.0,NaN,3.0,W,20.0,760.0,O,Brink's,NaN,1058868.0
1,LED1,LEDUC,TRUDY,NaN,F,245 congress court,APT. 624,77281,Quebec,PA,...,177967.0,5.0,1.0,U,3.0,555.0,U,NaN,6.0,1988185.0
2,lat2,latif,ireland,NaN,NaN,6517 frailing west,NaN,91355,Henderson,Alberta,...,321772.0,2.0,1.0,S,NaN,566.0,O,NaN,6.0,3673128.0
3,RUD3,RUDDICK,Misti,NaN,NaN,11449 blalock park,NaN,36615,Salem,AR,...,25449.0,2.0,1.0,W,77.0,NaN,O,Air Products & Chemicals,3.0,2005895.0
4,JAY4,JAYAMANNE,sarena,M,f,18354 dodd court,Apt. 619,t6a 1i3,Toledo,OR,...,166567.0,0.0,3.0,M,21.0,815.0,O,Oshkosh,4.0,624736.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11231,Luq11255,Luqwavkqptlr7a4mfe4ljxuvlrtbu,rqlwrqbacndgdsdalzypvu,NaN,F,4ndofwdqdrunlytfwvrg6mwuayujkzatcwtmzndkdklxl,PX5EPsJXIfFkWUFSLQXQT3SfIRTVPNFW,4oafz,Beaumomt,LA,...,143549.0,3.0,3.0,M,166.0,795.0,O,AbLAC,NaN,3410412.0
11232,Rgq11247,Rnqdvabnq,Vziqhemczksmjfnqrldgcliitgq,NaN,m,gxtaswortlwgiipindwzdmccjehkopxgkrnfujjgxbdyqd...,VYABDvBvII4DFIGFJU,Gjxyhdkgjpf,Hohston,NH,...,230648.0,1.0,2.0,M,60.0,264.0,O,IMSgealth,3.0,3522691.0
11233,cvtq1247,xvtqfsmr9nkw,zkcvgsgn0zopbsyvxohlawfcacsf,G,F,QmXMJCNSCJnVBXWBLzZXEYpMWORDLhNY5aOaYYZIUMCbEI...,TP5dAKPVWOINJFQVxH7AjIoJhsLPsJTQMFWVm3RHVSTFFI...,Vrbsklvlaa,Daloas,WY,...,255294.0,2.0,1.0,S,NaN,736.0,R,Ch3mtura,6.0,NaN
11234,ajf11248,ajbmfvboopntzlowpiodhbemitke9s,Koddgovylcrxbglkdjccchzeex,NaN,F,wjgewydbgwqdjbbclclwfzpuumklyuuddna3iavvuqxvjur,Fslqwnegoregjncju1fgubecchjvpwhtpxubbycz0bialp...,eqlhas,y3andRapids,Novxxcotia,...,294567.0,5.0,3.0,M,5.0,519.0,R,Dell,3.0,2099966.0


constraints and meta-information - вся доп инвормация об атрибутах, которая может помочь

In [2]:
def get_constraints(df_):
    constr = []
    for i in df_.columns:
        nnans = df_[i].isnull().sum()/len(df_[i])
        column = df_[i].dropna()
        type_ = column.dtype
        nunique = column.nunique()
        if type_=='object':
            max_len = column.apply(lambda x: len(x)).max()
            min_len = column.apply(lambda x: len(x)).min()
            average_len = column.apply(lambda x: len(x)).mean()
            describe = [0,0,0,0,0,0,0,0]
            t = 1
        else:
            max_len = 0
            min_len = 0
            average_len = 0
            describe = list(column.describe().values)
            t = 0
        
        constr.append([nnans,t,nunique,max_len,min_len,average_len]+describe)
    return constr

In [3]:
def get_data(df1,df2,ground_t1, data_name1='',data_name2=''):
    constr1 = get_constraints(df1)
    constr2 = get_constraints(df2)
    data = []
    for attr1_ind in range(len(df1.columns)):
        attr1 = df1.columns[attr1_ind]
        constr1_ = constr1[attr1_ind]
        match = (attr1,'')
        for ground in ground_t1:
            if attr1 == ground[0]:
                match = ground
        for attr2_ind in range(len(df2.columns)):
            attr2 = df2.columns[attr2_ind]
            constr2_ = constr2[attr2_ind]
            if attr2==match[1]:
                res=1
            else:
                res=0
            data.append([data_name1, data_name2, attr1,attr2,res,constr1_+constr2_])
    return data

In [6]:
data1 = get_data(df1,df2, ground_t1, data_name1=data_name1,data_name2=data_name2)

вторая пара

In [14]:
all_data_name2 = 'prospect_horizontal_50_ac5_ev'
data_name1 = all_data_name2+'_source'
data_name2 = all_data_name2+'_target'
df1 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}'.format(all_data_name2,data_name1+'.csv'))
df2 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}'.format(all_data_name2,data_name2+'.csv'))

with open(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}_mapping.json'.format(all_data_name2,all_data_name2), 'r') as f:
    mapping = json.load(f)
ground_t1 = []
for i in mapping['matches']:
    ground_t1.append((i['source_column'],i['target_column']))

data2 = get_data(df1,df2,ground_t1, data_name1=data_name1,data_name2=data_name2)

In [8]:
df_data= pd.DataFrame(data1+data2,columns = ['dataset1_name','dataset2_name', 'attr1_name', 'attr2_name', 'attribute_match', 'constraints'] )
df_data

,dataset1_name,dataset2_name,attr1_name,attr2_name,attribute_match,constraints
0,prospect_horizontal_50_ec_av_source,prospect_horizontal_50_ec_av_target,AgencyID,AgencyID,1,"[0.0, 1, 11236, 8, 4, 7.0107689569241725, 0, 0..."
1,prospect_horizontal_50_ec_av_source,prospect_horizontal_50_ec_av_target,AgencyID,LastName,0,"[0.0, 1, 11236, 8, 4, 7.0107689569241725, 0, 0..."
2,prospect_horizontal_50_ec_av_source,prospect_horizontal_50_ec_av_target,AgencyID,FirstName,0,"[0.0, 1, 11236, 8, 4, 7.0107689569241725, 0, 0..."
3,prospect_horizontal_50_ec_av_source,prospect_horizontal_50_ec_av_target,AgencyID,MiddleInitial,0,"[0.0, 1, 11236, 8, 4, 7.0107689569241725, 0, 0..."
4,prospect_horizontal_50_ec_av_source,prospect_horizontal_50_ec_av_target,AgencyID,Gender,0,"[0.0, 1, 11236, 8, 4, 7.0107689569241725, 0, 0..."
...,...,...,...,...,...,...
963,prospect_horizontal_50_ac5_ev_source,prospect_horizontal_50_ac5_ev_target,NetWorth,prospect_crdtrtng,0,"[0.05170879316482734, 0, 10640, 0, 0, 0, 10655..."
964,prospect_horizontal_50_ac5_ev_source,prospect_horizontal_50_ac5_ev_target,NetWorth,prospect_wnrrntflg,0,"[0.05170879316482734, 0, 10640, 0, 0, 0, 10655..."
965,prospect_horizontal_50_ac5_ev_source,prospect_horizontal_50_ac5_ev_target,NetWorth,prospect_mplr,0,"[0.05170879316482734, 0, 10640, 0, 0, 0, 10655..."
966,prospect_horizontal_50_ac5_ev_source,prospect_horizontal_50_ac5_ev_target,NetWorth,prospect_nmbrcrdtcrds,0,"[0.05170879316482734, 0, 10640, 0, 0, 0, 10655..."


In [9]:
len(df_data['constraints'][0])

28

In [10]:
df_data['attribute_match'].sum()

44

In [11]:
df_data.to_pickle('TPC-DI_1.p')

### Обучим smat+ с доп информацией и без нее

python train_2.py --dataset tpc --model_name aoa_3 --valset_ratio 0.1

best_model_path: state_dict_2/tpc/aoa_3_tpc_val_f1_0.9362

python train_2.py --dataset tpc --model_name aoa_2 --valset_ratio 0.1

best_model_path: state_dict_2/tpc/aoa_2_tpc_val_f1_0.9362

### Загрузим модели и проверим на данных, которые модели не видели

In [4]:
from time import strftime, localtime
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torchview import draw_graph

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight

from data_utils_2 import build_tokenizer, build_embedding_matrix, Dataset

from models import AOA, AOA_2, AOA_3, AOA_4 , AOA_5

In [11]:
class opt(object):
    def __init__(self):
        self.max_seq_len = 240
        self.dataset = 'tpc'
        self.embed_dim = 300
        self.hidden_dim = 300
        self.class_dim = 2
        self.constr_dim = 28
opt = opt()
fnames = ['./datasets/omap/TPC-DI_1.p']

tokenizer = build_tokenizer(
    fnames,
    max_seq_len=opt.max_seq_len,
    dat_fname='{0}_tokenizer.dat'.format(opt.dataset))
embedding_matrix = build_embedding_matrix(
    word2idx=tokenizer.word2idx,
    embed_dim=opt.embed_dim,
    dat_fname='{0}_{1}_embedding_matrix.dat'.format(str(opt.embed_dim), opt.dataset))

loading tokenizer: tpc_tokenizer.dat
loading embedding_matrix: 300_tpc_embedding_matrix.dat


In [14]:
model_1 = AOA_3(embedding_matrix ,opt)
model_1.load_state_dict(state_dict = torch.load('state_dict_2/tpc/aoa_3_tpc_val_f1_0.9362'))
model_1.eval()


AOA_3(
  (dropout): Dropout(p=0.1, inplace=False)
  (embed): Embedding(93, 300)
  (ctx_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (asp_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (ctxR_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (aspR_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=1800, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=2, bias=True)
  (lin1_c): Linear(in_features=28, out_features=64, bias=True)
  (bn1_c): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin2_c): Linear(in_features=64, out_features=64, bias=Tru

In [15]:
model_2 = AOA_2(embedding_matrix ,opt)
model_2.load_state_dict(state_dict = torch.load('state_dict_2/tpc/aoa_2_tpc_val_f1_0.9362'))
model_2.eval()

AOA_2(
  (dropout): Dropout(p=0.1, inplace=False)
  (embed): Embedding(93, 300)
  (ctx_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (asp_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (ctxR_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (aspR_lstm): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=1800, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=2, bias=True)
)

Загрузим тестовые данные

In [16]:
data_name1 = 'prospect_horizontal_50_ec_ev_source'
data_name2 = 'prospect_horizontal_50_ec_ev_target'
df1 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\prospect_horizontal_50_ec_ev\prospect_horizontal_50_ec_ev_source.csv')
df2 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\prospect_horizontal_50_ec_ev\prospect_horizontal_50_ec_ev_target.csv')

with open(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\prospect_horizontal_50_ec_ev\prospect_horizontal_50_ec_ev_mapping.json', 'r') as f:
    mapping = json.load(f)
ground_t1 = []
for i in mapping['matches']:
    ground_t1.append((i['source_column'],i['target_column']))

data_test1 = get_data(df1,df2,ground_t1, data_name1=data_name1,data_name2=data_name2)
df_data_test1= pd.DataFrame(data_test1,columns = ['dataset1_name','dataset2_name', 'attr1_name', 'attr2_name', 'attribute_match', 'constraints'] )
df_data_test1.to_pickle('test_1.p')

In [17]:
data_name3 = 'prospect_horizontal_50_ac1_ev_source'
data_name4 = 'prospect_horizontal_50_ac1_ev_target'
df3 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\prospect_horizontal_50_ac1_ev\prospect_horizontal_50_ac1_ev_source.csv')
df4 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\prospect_horizontal_50_ac1_ev\prospect_horizontal_50_ac1_ev_target.csv')

with open(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\prospect_horizontal_50_ac1_ev\prospect_horizontal_50_ac1_ev_mapping.json', 'r') as f:
    mapping = json.load(f)
ground_t2 = []
for i in mapping['matches']:
    ground_t2.append((i['source_column'],i['target_column']))

data_test2 = get_data(df3,df4,ground_t2, data_name1=data_name3,data_name2=data_name4)
df_data_test2= pd.DataFrame(data_test2,columns = ['dataset1_name','dataset2_name', 'attr1_name', 'attr2_name', 'attribute_match', 'constraints'] )
df_data_test2.to_pickle('test_2.p')

In [18]:
testset1 = Dataset('test_1.p', tokenizer,dat_fname='{0}_12_test.dat'.format(opt.dataset))

tpc
Finished write data file


In [19]:
testset2 = Dataset('test_2.p', tokenizer,dat_fname='{0}_13_test.dat'.format(opt.dataset))

tpc
Finished write data file


In [20]:
test_data_loader1 = DataLoader(dataset=testset1, batch_size=len(testset1), shuffle=False)
test_data_loader2 = DataLoader(dataset=testset2, batch_size=len(testset2), shuffle=False)

In [5]:
def _test_model(data_loader, m, inputs_cols, data_test):
    with torch.no_grad():
        for t_batch, t_sample_batched in enumerate(data_loader):
            t_inputs = [t_sample_batched[col] for col in inputs_cols]
            t_targets = t_sample_batched['class_n']
            t_outputs = m(t_inputs)
            res = []
            for i in range(len(t_targets)):
                if torch.argmax(t_outputs[i], -1)==1:
                    res.append((data_test[i][2:4],t_outputs[i]))
                    

            n_correct = (torch.argmax(t_outputs, -1) == t_targets).sum().item()
            n_total = len(t_outputs)

    acc = n_correct / n_total
    return acc, res


In [22]:
input_cols_1 = ['text_raw_indices1', 'aspect_indices1','text_raw_indices2', 'aspect_indices2', 'constraints']
res_1 = _test_model(test_data_loader1, model_1, input_cols_1, data_test1)
res_1[0]

0.9917355371900827

In [23]:
res_2 = _test_model(test_data_loader2, model_1, input_cols_1, data_test2)
res_2[0]

0.993801652892562

In [24]:
res_3 = _test_model(test_data_loader1, model_2, input_cols_1, data_test1)
res_3[0]

0.9958677685950413

In [25]:
res_4 = _test_model(test_data_loader2, model_2, input_cols_1, data_test2)
res_4[0]

0.9834710743801653

In [6]:
def _test_on_ground_truth(ground_tr, predicted):
    answer = {}
    
    predicted_unique = {i[0][1]:0 for i in predicted}
    
    for truth in ground_tr:
        chosen_pr=''
        for pr in predicted:
            if truth[0]==pr[0][0] and  predicted_unique[pr[0][1]]<1:
                if type(answer.get(truth[0], 0)) ==int: 
                    answer[truth[0]] = (pr[0][1], pr[1])
                    chosen_pr=pr[0][1]
                else:
                    prob_1 = nn.functional.softmax(answer[truth[0]][1], dim=0)
                    prob_2 = nn.functional.softmax(pr[1], dim=0)
                    if prob_1[1]<prob_2[1]:
                        answer[truth[0]] = (pr[0][1], pr[1])
                        chosen_pr=pr[0][1]
        predicted_unique[chosen_pr]=predicted_unique.get(chosen_pr,0)+1
    total_tr = 0
    for i in ground_tr:
        try:
            if i == (i[0], answer[i[0]][0]):
                total_tr+=1
        except:
            pass
    
    return total_tr/len(ground_tr), answer
                    

In [27]:
acc1 = _test_on_ground_truth(ground_t1, res_1[1])
acc2 = _test_on_ground_truth(ground_t2, res_2[1])
acc3 = _test_on_ground_truth(ground_t1, res_3[1])
acc4 = _test_on_ground_truth(ground_t2, res_4[1])

In [28]:
acc1[0],acc2[0],acc3[0],acc4[0] # AOA_3 data1, AOA_3 data2, AOA_2 data1, AOA_2 data2

(0.9090909090909091, 0.9090909090909091, 1.0, 0.7272727272727273)

In [29]:
acc1

(0.9090909090909091,
 {'AgencyID': ('AgencyID', tensor([-1.5182,  1.2892])),
  'LastName': ('LastName', tensor([-0.5781,  0.5512])),
  'FirstName': ('FirstName', tensor([-1.1674,  1.0134])),
  'MiddleInitial': ('MiddleInitial', tensor([-1.1680,  1.0006])),
  'Gender': ('Gender', tensor([-1.4718,  1.2285])),
  'AddressLine1': ('AddressLine2', tensor([-1.7490,  1.4741])),
  'AddressLine2': ('AddressLine1', tensor([-1.2272,  1.0650])),
  'PostalCode': ('PostalCode', tensor([-2.1963,  1.8275])),
  'City': ('City', tensor([-1.9069,  1.5723])),
  'State': ('State', tensor([-1.0223,  0.8825])),
  'Country': ('Country', tensor([-2.0826,  1.7179])),
  'Phone': ('Phone', tensor([-1.7757,  1.4942])),
  'Income': ('Income', tensor([-1.4526,  1.2180])),
  'NumberCars': ('NumberCars', tensor([-1.0707,  0.9685])),
  'NumberChildren': ('NumberChildren', tensor([-1.4229,  1.2414])),
  'MaritalStatus': ('MaritalStatus', tensor([-1.6027,  1.3469])),
  'Age': ('Age', tensor([-2.1089,  1.7785])),
  'Credit

AOA_3 спутала 'AddressLine1' и 'AddressLine2'. Остальное верно.

Модель без constraints может сравнивать атрибуты нормально только с похожими названиями

### Посмотрим результаты с использованием Coma

In [30]:
matcher = Coma(strategy="COMA_OPT")
matches_1 = valentine_match(df1, df2, matcher)#df1,df2,ground_t1
matches_2 = valentine_match(df3, df4, matcher)

In [7]:
def _test_on_ground_truth_valentine(ground_tr, predicted):
    i=0
    total_tr = 0
    for it in predicted.items():
        for gr in ground_tr:
            if gr == (it[0][0][1], it[0][1][1]):
                total_tr+=1
        i+=1
    return total_tr/len(ground_tr)

In [32]:
print(_test_on_ground_truth_valentine(ground_t1, matches_1))
print(_test_on_ground_truth_valentine(ground_t2, matches_2))

1.0
1.0


In [33]:
matches_2

{(('table_1', 'NumberCreditCards'),
  ('table_2', 'prospect_NumberCreditCards')): 0.686985,
 (('table_1', 'NumberChildren'),
  ('table_2', 'prospect_NumberChildren')): 0.6681818,
 (('table_1', 'MiddleInitial'),
  ('table_2', 'prospect_MiddleInitial')): 0.64880383,
 (('table_1', 'MaritalStatus'),
  ('table_2', 'prospect_MaritalStatus')): 0.64880383,
 (('table_1', 'OwnOrRentFlag'),
  ('table_2', 'prospect_OwnOrRentFlag')): 0.64880383,
 (('table_1', 'PostalCode'), ('table_2', 'prospect_PostalCode')): 0.63934183,
 (('table_1', 'AddressLine1'),
  ('table_2', 'prospect_AddressLine1')): 0.63687885,
 (('table_1', 'AddressLine2'),
  ('table_2', 'prospect_AddressLine2')): 0.63687885,
 (('table_1', 'CreditRating'),
  ('table_2', 'prospect_CreditRating')): 0.63687885,
 (('table_1', 'NumberCars'), ('table_2', 'prospect_NumberCars')): 0.60903883,
 (('table_1', 'FirstName'), ('table_2', 'prospect_FirstName')): 0.5926403,
 (('table_1', 'AgencyID'), ('table_2', 'prospect_AgencyID')): 0.5741608,
 (('tab

### Проверим на большем числе датасетов

In [8]:
def get_data_and_ground(all_data_name1):
    data_name1 = all_data_name1+'_source'
    data_name2 = all_data_name1+'_target'
    df1 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}'.format(all_data_name1,data_name1+'.csv'))
    df2 = pd.read_csv(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}'.format(all_data_name1,data_name2+'.csv'))

    with open(r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable\{}\{}_mapping.json'.format(all_data_name1,all_data_name1), 'r') as f:
        mapping = json.load(f)
    ground_t1 = []
    for i in mapping['matches']:
        ground_t1.append((i['source_column'],i['target_column']))
    return [data_name1,data_name2,df1,df2,ground_t1]

def test_dl_models(df_and_ground, model):
    data_name1= df_and_ground[0]
    data_name2= df_and_ground[1]
    df1 = df_and_ground[2]
    df2 = df_and_ground[3]
    ground_t1 = df_and_ground[4]
    
    name = data_name1 +'_'+ data_name2
    
    data_test1 = get_data(df1,df2,ground_t1, data_name1=data_name1,data_name2=data_name2)
    df_data_test1= pd.DataFrame(data_test1,columns = ['dataset1_name','dataset2_name', 'attr1_name', 'attr2_name', 'attribute_match', 'constraints'] )
    df_data_test1.to_pickle('test_{}.p'.format(name))
    
    testset1 = Dataset('test_{}.p'.format(name), tokenizer,dat_fname='{0}_12_test.dat'.format(opt.dataset))
    
    test_data_loader1 = DataLoader(dataset=testset1, batch_size=len(testset1), shuffle=False)
    
    input_cols_1 = ['text_raw_indices1', 'aspect_indices1','text_raw_indices2', 'aspect_indices2', 'constraints']
    
    res = _test_model(test_data_loader1, model, input_cols_1, data_test1)
    
    acc = _test_on_ground_truth(ground_t1, res[1])
        
    return acc

def test_valentine_models(df_and_ground, matcher):
    data_name1= df_and_ground[0]
    data_name2= df_and_ground[1]
    df1 = df_and_ground[2]
    df2 = df_and_ground[3]
    ground_t = df_and_ground[4]
    
    matches = valentine_match(df1, df2, matcher)
    return _test_on_ground_truth_valentine(ground_t, matches), matches
    

названия папок с датасетами на которых обучались

In [15]:
all_data_name1, all_data_name2

('prospect_horizontal_50_ec_av', 'prospect_horizontal_50_ac5_ev')

Прверим модели на всех данных кроме этих

In [36]:
pth = r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable'
dirs = os.listdir(pth)

model_1 = AOA_3(embedding_matrix ,opt)
model_1.load_state_dict(state_dict = torch.load('state_dict_2/tpc/aoa_3_tpc_val_f1_0.9362'))
model_1.eval()

model_2 = AOA_2(embedding_matrix ,opt)
model_2.load_state_dict(state_dict = torch.load('state_dict_2/tpc/aoa_2_tpc_val_f1_0.9362'))
model_2.eval()

model_3 = AOA_3(embedding_matrix ,opt)
model_3.load_state_dict(state_dict = torch.load('state_dict_2/tpc/aoa_3_tpc_val_f1_1.0'))
model_3.eval()

model_4 = AOA_3(embedding_matrix ,opt)
model_4.load_state_dict(state_dict = torch.load('state_dict_2/tpc/aoa_3_tpc_val_f1_0.9599'))
model_4.eval()

acc_models = {'AOA_3':[],'AOA_2':[],'COMA':[], 'AOA_3_cross':[], 'AOA_3_cross_2':[]}

for dir_ in dirs:
    matcher = Coma(strategy="COMA_OPT")
    if dir_ not in [all_data_name1, all_data_name2]:
        data = get_data_and_ground(dir_)
        
        acc1 = test_dl_models(data, model_1)
        acc2 = test_dl_models(data, model_2)
        acc3 = test_valentine_models(data, matcher)
        acc4 = test_dl_models(data, model_3)
        acc5 = test_dl_models(data, model_4)
        
        acc_models['AOA_3'] = acc_models['AOA_3']+[acc1[0]]
        acc_models['AOA_2'] = acc_models['AOA_2']+[acc2[0]]
        acc_models['COMA'] = acc_models['COMA']+[acc3[0]]
        acc_models['AOA_3_cross'] = acc_models['AOA_3_cross']+[acc4[0]]
        acc_models['AOA_3_cross_2'] = acc_models['AOA_3_cross_2']+[acc5[0]]
    
        
        

tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished w

In [37]:
np.mean(acc_models['AOA_3'])

0.5548128342245989

In [38]:
np.mean(acc_models['AOA_2'])

0.5762032085561497

In [39]:
np.mean(acc_models['COMA'])

0.7754010695187166

In [40]:
np.mean(acc_models['AOA_3_cross'])

0.7165775401069518

In [41]:
np.mean(acc_models['AOA_3_cross_2'])

0.6564171122994653

In [42]:
acc_models['AOA_3_cross']

[1.0,
 1.0,
 0.18181818181818182,
 0.22727272727272727,
 1.0,
 1.0,
 0.18181818181818182,
 0.18181818181818182,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.18181818181818182,
 0.18181818181818182,
 1.0,
 1.0,
 0.22727272727272727,
 0.18181818181818182,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.22727272727272727,
 0.22727272727272727,
 1.0,
 1.0,
 0.13636363636363635,
 0.22727272727272727,
 1.0,
 1.0]

In [43]:
acc_models['COMA']

[1.0,
 1.0,
 0.7727272727272727,
 0.7727272727272727,
 0.8636363636363636,
 0.8636363636363636,
 0.36363636363636365,
 0.36363636363636365,
 0.6363636363636364,
 0.6363636363636364,
 1.0,
 1.0,
 1.0,
 1.0,
 0.7727272727272727,
 0.8636363636363636,
 0.8636363636363636,
 0.8636363636363636,
 0.3181818181818182,
 0.4090909090909091,
 0.6363636363636364,
 0.6363636363636364,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8636363636363636,
 0.6818181818181818,
 0.8636363636363636,
 0.8636363636363636,
 0.36363636363636365,
 0.45454545454545453,
 0.6363636363636364,
 1.0]

In [44]:
new_dirs=[]
for dir_ in dirs:
    if dir_ not in [all_data_name1, all_data_name2]:
        new_dirs.append(dir_)

for i in range(len(new_dirs)):
    if new_dirs[i] in ['prospect_horizontal_50_ac1_ev', 'prospect_horizontal_50_ec_ev']:
        print(acc_models['AOA_3_cross'][i])

1.0
1.0


In [45]:
for i in range(len(new_dirs)):
    print(new_dirs[i], acc_models['AOA_3_cross'][i])

prospect_horizontal_0_ac1_av 1.0
prospect_horizontal_0_ac1_ev 1.0
prospect_horizontal_0_ac2_av 0.18181818181818182
prospect_horizontal_0_ac2_ev 0.22727272727272727
prospect_horizontal_0_ac3_av 1.0
prospect_horizontal_0_ac3_ev 1.0
prospect_horizontal_0_ac4_av 0.18181818181818182
prospect_horizontal_0_ac4_ev 0.18181818181818182
prospect_horizontal_0_ac5_av 1.0
prospect_horizontal_0_ac5_ev 1.0
prospect_horizontal_0_ec_av 1.0
prospect_horizontal_0_ec_ev 1.0
prospect_horizontal_100_ac1_av 1.0
prospect_horizontal_100_ac1_ev 1.0
prospect_horizontal_100_ac2_av 0.18181818181818182
prospect_horizontal_100_ac2_ev 0.18181818181818182
prospect_horizontal_100_ac3_av 1.0
prospect_horizontal_100_ac3_ev 1.0
prospect_horizontal_100_ac4_av 0.22727272727272727
prospect_horizontal_100_ac4_ev 0.18181818181818182
prospect_horizontal_100_ac5_av 1.0
prospect_horizontal_100_ac5_ev 1.0
prospect_horizontal_100_ec_av 1.0
prospect_horizontal_100_ec_ev 1.0
prospect_horizontal_50_ac1_av 1.0
prospect_horizontal_50_ac1

In [46]:
df_and_ground = get_data_and_ground('prospect_horizontal_0_ac2_av')
data_name1= df_and_ground[0]
data_name2= df_and_ground[1]
df1 = df_and_ground[2]
df2 = df_and_ground[3]
ground_t1 = df_and_ground[4]
    
data_test1 = get_data(df1,df2,ground_t1, data_name1=data_name1,data_name2=data_name2)
df_data_test1= pd.DataFrame(data_test1,columns = ['dataset1_name','dataset2_name', 'attr1_name', 'attr2_name', 'attribute_match', 'constraints'] )
df_data_test1.to_pickle('test_{}.p'.format(all_data_name1))
    
testset1 = Dataset('test_{}.p'.format(all_data_name1), tokenizer,dat_fname='{0}_12_test.dat'.format(opt.dataset))
    
test_data_loader1 = DataLoader(dataset=testset1, batch_size=len(testset1), shuffle=False)
    
input_cols_1 = ['text_raw_indices1', 'aspect_indices1','text_raw_indices2', 'aspect_indices2', 'constraints']
    
res = _test_model(test_data_loader1, model_3, input_cols_1, data_test1)

tpc
Finished write data file


In [47]:
ground_t1

[('AgencyID', 'AgeID'),
 ('LastName', 'LaN'),
 ('FirstName', 'FiN'),
 ('MiddleInitial', 'MidInit'),
 ('Gender', 'Ge'),
 ('AddressLine1', 'AdL1'),
 ('AddressLine2', 'AdLi2'),
 ('PostalCode', 'PosC'),
 ('City', 'Ci'),
 ('State', 'Sta'),
 ('Country', 'Coun'),
 ('Phone', 'Ph'),
 ('Income', 'Inc'),
 ('NumberCars', 'NumC'),
 ('NumberChildren', 'NuCh'),
 ('MaritalStatus', 'MariSta'),
 ('Age', 'Ag'),
 ('CreditRating', 'CrRat'),
 ('OwnOrRentFlag', 'OOReFl'),
 ('Employer', 'Em'),
 ('NumberCreditCards', 'NuCreCa'),
 ('NetWorth', 'NeWo')]

In [48]:
res

(0.9442148760330579,
 [(['AddressLine1', 'AdL1'], tensor([-0.4321,  0.4330])),
  (['AddressLine2', 'AdLi2'], tensor([-1.3647,  1.3624])),
  (['City', 'OOReFl'], tensor([-1.7710,  1.7691])),
  (['Phone', 'LaN'], tensor([-0.1798,  0.1812])),
  (['Phone', 'FiN'], tensor([-0.1798,  0.1812])),
  (['Phone', 'MidInit'], tensor([-0.0269,  0.0278])),
  (['Phone', 'Ph'], tensor([-2.2842,  2.2812])),
  (['Phone', 'Inc'], tensor([-0.1466,  0.1479])),
  (['Phone', 'NuCh'], tensor([-0.4969,  0.4960])),
  (['Phone', 'Em'], tensor([-0.0508,  0.0519])),
  (['Phone', 'NuCreCa'], tensor([-0.4916,  0.4907])),
  (['MaritalStatus', 'MariSta'], tensor([-0.4891,  0.4900])),
  (['Age', 'AgeID'], tensor([-0.1608,  0.1609]))])

In [49]:
acc = _test_on_ground_truth(ground_t1, res[1])
acc

(0.18181818181818182,
 {'AddressLine1': ('AdL1', tensor([-0.4321,  0.4330])),
  'AddressLine2': ('AdLi2', tensor([-1.3647,  1.3624])),
  'City': ('OOReFl', tensor([-1.7710,  1.7691])),
  'Phone': ('Ph', tensor([-2.2842,  2.2812])),
  'MaritalStatus': ('MariSta', tensor([-0.4891,  0.4900])),
  'Age': ('AgeID', tensor([-0.1608,  0.1609]))})

### А что если обучить на 3 датасетах?

In [9]:
def create_train_data(names):
    data=[]
    for name in names:
        df_and_ground = get_data_and_ground(name)

        data_name1= df_and_ground[0]
        data_name2= df_and_ground[1]
        df1 = df_and_ground[2]
        df2 = df_and_ground[3]
        ground_t1 = df_and_ground[4]

        data_train = get_data(df1,df2,ground_t1, data_name1=data_name1,data_name2=data_name2)
        data+=data_train
    df_data= pd.DataFrame(data,columns = ['dataset1_name','dataset2_name', 'attr1_name', 'attr2_name', 'attribute_match', 'constraints'] )
    df_data.to_pickle('./datasets/omap/train_tpc.p')
names = ['prospect_horizontal_50_ec_ev','prospect_horizontal_50_ac2_ev','prospect_horizontal_50_ac5_av']


In [ ]:
create_train_data(names)

Обучаем модель

python train_2.py --dataset tpc_2 --model_name aoa_3

state_dict_2/tpc_2/aoa_3_tpc_2_val_f1_0.9755

In [51]:

pth = r'C:\Users\shepe\Downloads\Valentine-datasets\prospect\Unionable'
dirs = os.listdir(pth)

class opt(object):
    def __init__(self):
        self.max_seq_len = 240
        self.dataset = 'tpc_2'
        self.embed_dim = 300
        self.hidden_dim = 300
        self.class_dim = 2
        self.constr_dim = 28
opt = opt()
fnames = ['./datasets/omap/train_tpc.p']

tokenizer = build_tokenizer(
    fnames,
    max_seq_len=opt.max_seq_len,
    dat_fname='{0}_tokenizer.dat'.format(opt.dataset))
embedding_matrix = build_embedding_matrix(
    word2idx=tokenizer.word2idx,
    embed_dim=opt.embed_dim,
    dat_fname='{0}_{1}_embedding_matrix.dat'.format(str(opt.embed_dim), opt.dataset))

model_1 = AOA_3(embedding_matrix ,opt)
model_1.load_state_dict(state_dict = torch.load('state_dict_2/tpc_2/aoa_3_tpc_2_val_f1_0.9755'))#10 эпох
model_1.eval()

model_2 = AOA_3(embedding_matrix ,opt)
model_2.load_state_dict(state_dict = torch.load('state_dict_2/tpc_2/aoa_3_tpc_2_val_f1_0.9762'))#10+ эпох
model_2.eval()

acc_models = {'AOA_3_cross':[], 'AOA_3_cross_2':[],'COMA':[], 'Cupid':[], 'DistributionBased':[]}

for dir_ in dirs:
    matcher_1 = Coma(strategy="COMA_OPT")
    matcher_2 = Cupid()
    matcher_3 = DistributionBased()
    if dir_ not in names:
        data = get_data_and_ground(dir_)
        
        acc1 = test_dl_models(data, model_1)
        acc2 = test_dl_models(data, model_2)
        acc3 = test_valentine_models(data, matcher_1)
        acc4 = test_valentine_models(data, matcher_2)
        acc5 = test_valentine_models(data, matcher_3)
        
        acc_models['AOA_3_cross'] = acc_models['AOA_3_cross']+[acc1[0]]
        acc_models['AOA_3_cross_2'] = acc_models['AOA_3_cross_2']+[acc2[0]]
        acc_models['COMA'] = acc_models['COMA']+[acc3[0]]
        acc_models['Cupid'] = acc_models['Cupid']+[acc4[0]]
        acc_models['DistributionBased'] = acc_models['DistributionBased']+[acc5[0]]

loading tokenizer: tpc_2_tokenizer.dat
loading embedding_matrix: 300_tpc_2_embedding_matrix.dat
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
Finished write data file
tpc
F

In [52]:
np.mean(acc_models['AOA_3_cross'])

0.8402203856749311

In [53]:
np.mean(acc_models['AOA_3_cross_2'])

0.8333333333333334

In [54]:
np.mean(acc_models['COMA'])

0.778236914600551

In [55]:
np.mean(acc_models['Cupid'])

0.5840220385674931

In [56]:
np.mean(acc_models['DistributionBased'])

0.7782369146005509